In [1]:
# 1) ensure we’re in the repo root
%cd ~/hard_disk/exoTEDRF_fork
!pwd
!ls

# 2) add repo-root to PYTHONPATH
import sys, os
sys.path.insert(0, os.getcwd())


/data/pascal/exoTEDRF_fork


/data/pascal/exoTEDRF_fork/exoenv_fork/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/data/pascal/exoTEDRF_fork/exoenv_fork/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/data/pascal/exoTEDRF_fork
changelog.md		  exotedrf		 Optimize_WASP39b
Check_install_fork.ipynb  exotedrf.egg-info	 pipeline_outputs_directory
Cost_function_5h.txt	  __init__.py		 README.md
Cost_function.txt	  joss			 readthedocs.yaml
docs			  LICENSE		 setup.py
exoenv_fork		  optimize_stage1.ipynb  WASP39b_test


In [2]:
import os

# point CRDS at your Optimize_WASP39b cache
os.environ['CRDS_PATH']       = os.path.join(os.getcwd(), 'Optimize_WASP39b', 'crds_cache')
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
# force CRDS to use your locally‐cached pmap
os.environ['CRDS_CONTEXT']    = 'jwst_1322.pmap'

print("CRDS_PATH:   ", os.environ['CRDS_PATH'])
print("CRDS_SERVER_URL:", os.environ['CRDS_SERVER_URL'])
print("CRDS_CONTEXT:", os.environ['CRDS_CONTEXT'])


CRDS_PATH:    /data/pascal/exoTEDRF_fork/Optimize_WASP39b/crds_cache
CRDS_SERVER_URL: https://jwst-crds.stsci.edu
CRDS_CONTEXT: jwst_1322.pmap


In [3]:
"""# Cell 2: open & slice the first 50 ints of segment 1
from jwst import datamodels
from exotedrf.utils import parse_config

# Load the YAML from Optimize_WASP39b/
cfg = parse_config("Optimize_WASP39b/run_WASP39b.yaml")

# Full path into the DMS_uncal folder
seg1 = "Optimize_WASP39b/DMS_uncal/jw01366003001_04101_00001-seg001_nrs1_uncal.fits"
print("seg1 →", seg1)

# Open and slice
dm_full = datamodels.open(seg1)
K = 50
dm_slice = dm_full.copy()
dm_slice.data = dm_full.data[:K]
dm_slice.meta.exposure.nints = K
dm_full.close()

print(f"Sliced → dm_slice.data.shape = {dm_slice.data.shape}")
print(f"Meta nints = {dm_slice.meta.exposure.nints}")
"""

'# Cell 2: open & slice the first 50 ints of segment 1\nfrom jwst import datamodels\nfrom exotedrf.utils import parse_config\n\n# Load the YAML from Optimize_WASP39b/\ncfg = parse_config("Optimize_WASP39b/run_WASP39b.yaml")\n\n# Full path into the DMS_uncal folder\nseg1 = "Optimize_WASP39b/DMS_uncal/jw01366003001_04101_00001-seg001_nrs1_uncal.fits"\nprint("seg1 →", seg1)\n\n# Open and slice\ndm_full = datamodels.open(seg1)\nK = 50\ndm_slice = dm_full.copy()\ndm_slice.data = dm_full.data[:K]\ndm_slice.meta.exposure.nints = K\ndm_full.close()\n\nprint(f"Sliced → dm_slice.data.shape = {dm_slice.data.shape}")\nprint(f"Meta nints = {dm_slice.meta.exposure.nints}")\n'

In [4]:
"""# Cell 3: time Stage 1 on the 50-int slice
import time
from exotedrf.stage1 import run_stage1

# Run with skip_steps=['OneOverFStep','JumpStep']
t0 = time.perf_counter()
result = run_stage1(
    [dm_slice],
    mode=cfg['observing_mode'],
    baseline_ints=dm_slice.meta.exposure.nints,
    save_results=False,
    skip_steps=['OneOverFStep','JumpStep'],
    nirspec_mask_width=cfg['nirspec_mask_width'],
    **cfg['stage1_kwargs']
)
t1 = time.perf_counter()

print(f"→ Stage 1 on 50 ints took {t1 - t0:.1f} s")
"""

'# Cell 3: time Stage 1 on the 50-int slice\nimport time\nfrom exotedrf.stage1 import run_stage1\n\n# Run with skip_steps=[\'OneOverFStep\',\'JumpStep\']\nt0 = time.perf_counter()\nresult = run_stage1(\n    [dm_slice],\n    mode=cfg[\'observing_mode\'],\n    baseline_ints=dm_slice.meta.exposure.nints,\n    save_results=False,\n    skip_steps=[\'OneOverFStep\',\'JumpStep\'],\n    nirspec_mask_width=cfg[\'nirspec_mask_width\'],\n    **cfg[\'stage1_kwargs\']\n)\nt1 = time.perf_counter()\n\nprint(f"→ Stage 1 on 50 ints took {t1 - t0:.1f} s")\n'

In [5]:
# from your notebook, still in /data/pascal/exoTEDRF_fork
!find . -maxdepth 2 -name "optimize.py"


./exotedrf/optimize.py


In [6]:
# Assuming it’s under exotedrf/
!python3 exotedrf/optimize.py --config Optimize_WASP39b/run_WASP39b.yaml



→ Optimizing time_window (others fixed = {'box_size': 80, 'thresh': 10, 'rejection_threshold': 10, 'time_rejection_threshold': 10, 'nirspec_mask_width': 15, 'w1': 1.0, 'w2': 1.0, 'w3': 1.0})
Running with params: {'time_window': 5, 'box_size': 80, 'thresh': 10, 'rejection_threshold': 10, 'time_rejection_threshold': 10, 'nirspec_mask_width': 15, 'w1': 1.0, 'w2': 1.0, 'w3': 1.0}
→ about to call run_stage1 with jump_kwargs: {'window': 5, 'box_size': 80}
start running Stage 1 from Fork
2025-07-09 10:03:27.110 - exoTEDRF - INFO - **Starting exoTEDRF Stage 1**
2025-07-09 10:03:27.110 - exoTEDRF - INFO - Detector level processing
2025-07-09 10:03:27.114 - exoTEDRF - WARNING - Setting "save_results=False" can be memory intensive.
2025-07-09 10:03:27,116 - stpipe.DQInitStep - INFO - DQInitStep instance created.
2025-07-09 10:03:28,869 - stpipe.DQInitStep - INFO - DQInitStep instance created.
2025-07-09 10:03:29,077 - stpipe.DQInitStep - INFO - Step DQInitStep running with args (<Level1bModel(60

In [7]:
from IPython.display import Javascript

def notify():
    display(Javascript('alert("Notebook cell finished!")'))

notify()

<IPython.core.display.Javascript object>